In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

##### Runnable Passthrough

It does not do anything to the input data.

Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [2]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [3]:
chain.invoke("Amar")

'Amar'

##### Runnable Lambda

To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.

Let's define a very simple function to create Russian lastnames:



In [4]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [5]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [6]:
chain.invoke("Amar")

'Amarovich'

##### Runnable Parallel

We will use RunnableParallel() for running tasks in parallel.

This is probably the most important and most useful Runnable from LangChain.

In the following chain, RunnableParallel is going to run these two tasks in parallel:
   * operation_a will use RunnablePassthrough.
   * operation_b will use RunnableLambda with the russian_lastname function.

In [7]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [8]:
chain.invoke("Amar")

{'operation_a': 'Amar', 'operation_b': 'Amarovich'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [ ]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [ ]:
chain = RunnableParallel(
    #inside the runnable parallel, i gave two tasks
    {
        "operation_a": RunnablePassthrough(), #it returns the same input
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary), #take another function and perform some operation
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [ ]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})